In [1]:
from dotenv import load_dotenv
import os
from langchain.chains import AnalyzeDocumentChain
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from datetime import datetime
import pymysql
from pymysql import Error

In [2]:
load_dotenv('./var.env')

api_key = os.getenv('OPENAI_API_KEY')

In [3]:

llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=api_key)

qa_chain = load_qa_chain(llm, chain_type="map_reduce")

qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

with open("./pregunta.txt") as f:
    file = f.read()

# Obtener la fecha y hora actuales
fecha_hora_actual = datetime.now()
fecha_hora_formateada = fecha_hora_actual.strftime('%Y-%m-%d %H:%M:%S')

question="Who is this guy? give me a resume about his stuff answer me in spanish, no more than 20 words"

response = qa_document_chain.run(
    input_document=file,
    question=question,
)

print(response)

# Crear un registro con la pregunta, respuesta y fecha y hora
registro = {
    "pregunta": question,
    "respuesta": response,
    "fecha_hora": fecha_hora_formateada
}

# Imprimir el registro
registro

Hugo Chávez, político y militar venezolano, presidente de Venezuela desde 1999 hasta su fallecimiento en 2013.


{'pregunta': 'Who is this guy? give me a resume about his stuff answer me in spanish, no more than 20 words',
 'respuesta': 'Hugo Chávez, político y militar venezolano, presidente de Venezuela desde 1999 hasta su fallecimiento en 2013.',
 'fecha_hora': '2023-12-17 14:34:48'}

In [22]:

def leer_archivo(ruta_archivo):
    try:
        with open(ruta_archivo) as f:
            return f.read()
    except Exception as e:
        print(f"Error al leer el archivo: {e}")
        return None

def obtener_fecha_hora_actual():
    try:
        fecha_hora_actual = datetime.now()
        return fecha_hora_actual.strftime('%Y-%m-%d %H:%M:%S')
    except Exception as e:
        print(f"Error al obtener la fecha y hora: {e}")
        return None

def procesar_documento(api_key, ruta_archivo, question):
    try:
        llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=api_key)
        qa_chain = load_qa_chain(llm, chain_type="map_reduce")
        qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

        documento = leer_archivo(ruta_archivo)
        if documento is None:
            return None

        response = qa_document_chain.run(input_document=documento, question=question)
        return response
    except Exception as e:
        print(f"Error al procesar el documento: {e}")
        return None

# Ejemplo de uso
ruta_archivo = "./pregunta.txt"
question = "Who is this guy? give me a resume about his stuff answer me in spanish, no more than 20 words"

respuesta = procesar_documento(api_key, ruta_archivo, question)
fecha_hora_formateada = obtener_fecha_hora_actual()

if respuesta is not None and fecha_hora_formateada is not None:
    registro = {
        "pregunta": question,
        "respuesta": respuesta,
        "fecha_hora": fecha_hora_formateada
    }
    print(registro)


{'pregunta': 'Who is this guy? give me a resume about his stuff answer me in spanish, no more than 20 words', 'respuesta': 'Hugo Chávez fue un político y militar venezolano, presidente de Venezuela desde 1999 hasta su fallecimiento en 2013.', 'fecha_hora': '2023-12-17 14:47:53'}


## FUNCIONES

In [18]:


def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = pymysql.connect(
            host=host_name,
            user=user_name,
            password=user_password
        )
        print("Conexión al servidor de MySQL exitosa")
    except Error as e:
        print(f"El error '{e}' ocurrió")
    return connection

def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = pymysql.connect(
            host=host_name,
            user=user_name,
            password=user_password,
            database=db_name
        )
        print("Conexión a la base de datos exitosa")
    except Error as e:
        print(f"El error '{e}' ocurrió")
    return connection

def execute_query(connection, query):
    with connection.cursor() as cursor:
        try:
            cursor.execute(query)
            connection.commit()
            print("Consulta ejecutada con éxito")
        except Error as e:
            print(f"El error '{e}' ocurrió")
            
def create_database(connection, query):
    with connection.cursor() as cursor:
        try:
            cursor.execute(query)
            connection.commit()
            print("Base de datos creada con éxito")
        except Error as e:
            print(f"El error '{e}' ocurrió")

       



## QUERYS

In [16]:
create_database_query = "CREATE DATABASE IF NOT EXISTS bbdd_gpt"     

create_usuario_table = """
CREATE TABLE IF NOT EXISTS Usuario (
  id_usuario INT AUTO_INCREMENT PRIMARY KEY,
  nombre VARCHAR(255) NOT NULL
);
"""

create_pregunta_table = """
CREATE TABLE IF NOT EXISTS Pregunta (
  id_pregunta INT AUTO_INCREMENT PRIMARY KEY,
  id_usuario INT,
  fecha_pregunta DATE NOT NULL,
  texto_pregunta VARCHAR(255) NOT NULL,
  FOREIGN KEY (id_usuario) REFERENCES Usuario(id_usuario)
);
"""

create_fecha_table = """
CREATE TABLE IF NOT EXISTS Fecha (
  id_fecha INT AUTO_INCREMENT PRIMARY KEY,
  id_pregunta INT,
  fecha_pregunta DATE NOT NULL,
  fecha_respuesta DATE NOT NULL,
  FOREIGN KEY (id_pregunta) REFERENCES Pregunta(id_pregunta)
);
"""

create_respuesta_table = """
CREATE TABLE IF NOT EXISTS Respuesta (
  id_respuesta INT AUTO_INCREMENT PRIMARY KEY,
  id_pregunta INT,
  texto_respuesta VARCHAR(255) NOT NULL,
  id_fecha INT,
  FOREIGN KEY (id_pregunta) REFERENCES Pregunta(id_pregunta),
  FOREIGN KEY (id_fecha) REFERENCES Fecha(id_fecha)
);
"""


In [19]:
connection = create_server_connection("127.0.0.1", "danie", "123456")

# Crear la base de datos
create_database(connection, create_database_query)

# Conectar a la base de datos específica
db_connection = create_db_connection("127.0.0.1", "danie", "123456", "bbdd_gpt")
connection.close()

Conexión al servidor de MySQL exitosa
Base de datos creada con éxito
Conexión a la base de datos exitosa


In [21]:
execute_query(connection, create_usuario_table)
execute_query(connection, create_pregunta_table)
execute_query(connection, create_fecha_table)
execute_query(connection, create_respuesta_table)

# # No olvide cerrar la conexión cuando haya terminado
# connection.close()

El error '(0, '')' ocurrió
El error '(0, '')' ocurrió
El error '(0, '')' ocurrió
El error '(0, '')' ocurrió
